# Create folder directories - skip if done

TODO turn this into a Python file or utils function

This is useful if you are using [Ceph](https://ceph.com/en/)

In [ ]:
!ls ~/workspace/turb

In [ ]:
%cd ~/workspace/turb/data01_01

In [ ]:
# creatuple list of folder on filedb to which data should be copied
# you may want to change this to avoid certain volumes
folders=[f'/home/idies/workspace/turb/data{str(f).zfill(2)}_{str(d).zfill(2)}/zarr/ncar-zarr/'  for f in range(1,13) for d in range(1,4)]
folders[:5]

In [ ]:
# Avoiding 7-2 and 9-2 - they're too full as of May 2023
# folders.remove("/home/idies/workspace/turb/data02_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data09_02/zarr/ncar-zarr/")
folders.remove("/home/idies/workspace/turb/data07_02/zarr/ncar-zarr/") # This is already created

In [ ]:
# Chatgpt - create folders above - skip if already created

import os

def create_folders(paths):
    for path in paths:
        os.makedirs(path, exist_ok=True)
        print(f"Folder created at {path}")

# create_folders(folders)


In [ ]:
# create dataXX_XX/zarr/ncar-zarr/timestep
timesteps = 6

timestep_folders = [f + "00" + str(i) for i in range(timesteps) for f in folders]
timestep_folders

In [ ]:
# Create subfolders - skip if done already

# create_folders(timestep_folders)

In [ ]:
variable_names_set = {"e", "t", "p", "u", "v", "w"}

# create dataXX_XX/zarr/ncar-zarr/timestep/Variable_name
variable_folders = [t + "/" + value for index, value in enumerate(variable_names_set) for t in timestep_folders]
variable_folders[:20]

In [ ]:
variable_names_set = {"e", "t", "p", "u", "v", "w"}

dest_directory_dict = {}

for index, value in enumerate(variable_names_set):
    dest_directory_dict[value] = [t + "/" + value for t in timestep_folders]

dest_directory_dict

In [ ]:
# create dataXX_XX/zarr/ncar-zarr/timestep/Variable_name
# create_folders(variable_folders)

# Write to folders

In [ ]:
%cd /home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/jhd-000-nc-zarr/e

In [ ]:
file_list = [file for file in os.listdir() if not file.startswith('.')]
len(file_list)

In [ ]:
%pwd

In [ ]:
import shutil

source_directory = '/home/idies/workspace/turb/data02_02/ariel-6-timestep-ncar-zarr/'
timestep_prefix = "jhd-{t}-nc-zarr"

# [timestep_prefix.format(t=t) for t in range(timesteps)]

# For each timestep
for t in range(timesteps):
    padded_timestep_index = str(t).zfill(3) # Paths are 001,002, not 1,2,3...
    
    timestep_dir = timestep_prefix.format(t=padded_timestep_index)
    
    print("Working on " + timestep_dir)
    
    # For each variable name
    for index, varname in enumerate(variable_names_set):
        pwd = source_directory + timestep_dir + "/" + varname
        print("Current working dir: ", pwd)
        
        # Don't move zarr metadata. TODO deal with this later
        source_file_paths = [file for file in os.listdir(pwd) if not file.startswith('.')]
        
        # Select only target dirs for current var and timestep
        dest_file_paths = [string for string in variable_folders if string.endswith(padded_timestep_index + "/" + varname)]
        
#         print(dest_file_paths)
        
        
        # for each zarr file
        for i in range(len(source_file_paths)):
#             shutil.move(source_file_paths[i], dest_file_paths[i % len(dest_file_paths)])
            try:
                shutil.copy(source_file_paths[i], dest_file_paths[i % len(dest_file_paths)])
            except Error:
                print(f"File '{dest_file_paths[i % len(dest_file_paths)]}' already exists. Skipping.")
#             shutil.move(source_file_paths[i], dest_file_paths[i % len(dest_file_paths)])


# <font color="red"> TODO Fix Zarr Metadata </font>

In [ ]:
z = zarr.open("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr")
z.info

In [ ]:
# zarr.consolidate_metadata("/home/idies/workspace/turb/data02_03/zarr/ncar-zarr/e")